# Shuffle a List

A simple algorithm that wasn't obvious to me, until an AI showed it to me. :facepalm:

Below is a naive shuffle algorithm. It randomly suffles the elements of a Python list. It's a bad implementation.

As an exercise, look it over and make some educated guesses as to why this won't work well.

In [ ]:
import random

def bad_shuffle(arr):

    # decide ahead of time how many times to swap elements
    # Is this enough swaps to really really shuffle the list?
    count = int(len(arr))
    # LATER, ADJUST THIS CONSTANT
    count = int(1.0 * count)
    
    for _ in range(1, count):
        # pick a list index at rantom
        i = random.randint(0, len(arr) - 1)
        # pick another index at random
        j = random.randint(0, len(arr) - 1)
        # swap the contents at the two indexes in a fancy way
        arr[i], arr[j] = arr[j], arr[i]
    return arr


## Exercise:

Edit this box and record your thoughts here.

**IMPORTANT:** It's fine to get these wrong. Guess!

What exactly do you think will go wrong when we use this algorithm?
 
Why is that?

## Spoilers

Here are some correct answers. This naive shuffle algorithm has several weaknesses:

1. **Non-uniform distribution:** Since the algorithm chooses two random indices i and j for each iteration and simply swaps their values, there's a higher chance that some elements will remain in their original positions, or the resulting distribution will be non-uniform. A good shuffle algorithm should uniformly distribute the elements across all positions.

2. **Limited number of swaps:** The algorithm only performs a number of swaps equal to the length of the array. This may not be enough to fully shuffle the array. The Knuth shuffle, for example, performs a number of swaps equal to the length of the array minus one but iterates from the end to the beginning, ensuring a more thorough shuffle.

3. **Inefficient:** In this algorithm, the number of iterations is directly proportional to the length of the array. Although this might seem efficient, the uniformity of the shuffle is not guaranteed, and increasing the number of iterations may not necessarily result in a more uniform distribution.

To overcome these weaknesses, we can use the Knuth shuffle (Fisher-Yates shuffle) algorithm, which provides a uniform distribution and is more efficient.

But before we do that, let's try to *visualize* this shuffle and maybe we can see with our eyes what is wrong.

Run this single shuffle visualizer a bunch of times. Can you see any of these weaknesses in the output?

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot_shuffle(arr, shuffle_fn):
    shuffled_arr = shuffle_fn(arr.copy())
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
    
    ax1.bar(range(len(arr)), arr, color='b')
    ax1.set_title("Original Array")
    ax1.set_xlabel("Index")
    ax1.set_ylabel("Value")
    
    ax2.bar(range(len(shuffled_arr)), shuffled_arr, color='r')
    ax2.set_title("Shuffled Array")
    ax2.set_xlabel("Index")
    ax2.set_ylabel("Value")
    
    plt.show()

arr = list(range(1, 50))
plot_shuffle(arr, bad_shuffle)


When I look at this output with my own eyes, most of the shuffled arrays look a lot like the original arrays.

It looks like they are only parially shuffled.

I can sort of detect bias visually from this graph. However, this is not a clear indicator of bad performance.

## Exercise

Go back to the `bad_shuffle` algorithm and modify it to make more swaps. Try multiplying the `count` variable by 2 or more to see if you can get better looking results.

Try changing the `list(range(...))` expression to make a longer array. Can you see the bias with fewer swaps?

Mutliplying by a really large number obviously helps. Why is this not a good solution? (It's not a trick question.)

About how many swaps are needed before the bias isn't detectable?

## A Better Visualization

Below we have a better visualization which produces a heatmap.

It runs the shuffle many times.

There is a stacked bar for each index  of the array. Vertically, boxes are colored to show how often a particular number appeared at that index.

Yellow is bad in this visualization, and blue to blue-green is good. Yellow indicates that the same value appeared at an index at least twice as often as we would have expected.

**IMPORTANT:** Reset the `bad_shuffle` function so the count variable is equal to the length of the array.

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

def shuffle_counts(arr, num_shuffles, shuffle_fn):
    counts = np.zeros((len(arr), len(arr)))

    for _ in range(num_shuffles):
        shuffled_arr = shuffle_fn(arr.copy())
        for i, value in enumerate(shuffled_arr):
            counts[i][value-1] += 1

    return counts

def plot_colored_histogram(arr, num_shuffles, shuffle_fn):
    counts = shuffle_counts(arr, num_shuffles, shuffle_fn)
    expected = num_shuffles / len(arr)
    vmin = expected / 2
    vmax = expected * 2
    

    fig, ax = plt.subplots(figsize=(10, 6))

    im=ax.pcolormesh(counts, vmin = vmin, vmax = vmax)
    ax.set_title("Colored Histogram")
    ax.set_xlabel("Index")
    ax.set_ylabel("Values Found")

    fig.colorbar(im, ax=ax)
    
    plt.show()

arr = list(range(1, 26))
num_shuffles = 10000
plot_colored_histogram(arr, num_shuffles, bad_shuffle)



## A Curious Diagonal Line

If things are going well, at this point we have a bright diagonal line on the color map.

This is telling us that at index one, the first element in the original array appears here more often than elsewhere. At index two, the second element of the original array appears here more often than elsewhere.

It showing us clearly, the problem that some elements never get randomly selected for swapping.

## Exercise

1. Adjust the `bad_shuffle` function again to increase the number of swaps. How much do you have to increase the swap count before the bias become undetectable?
2. What does a better algorithm need to do to eliminate this bias?

## The Correct Answer

Below is the Knuth or Fisher-Yates algorithm. Study it a bit and contrast it with the bad algorithm.

In [ ]:
import random

def knuth_shuffle(arr):
    for i in range(len(arr) - 1, 0, -1):
        # Pick a random index
        j = random.randint(0, i)
        # Swap the contents of that random index with this one.
        arr[i], arr[j] = arr[j], arr[i]
    return arr


## Exercise

1. Assume this algorithm is correct. How fast is it compared to `bad_shuffle`?
2. How does it manage to completely eliminate bias? (Hint: why was the old algorithm biased?)

## Spoilers

1. It is more or less equivalent to `bad_shuffle` when count is equal to the length of the list. However, it manages to be correct in that time.
2. Every element is guaranteed to be swapped at least once, which was the flaw in `bad_shuffle`

## Visualize the Better Shuffle

Let's start with the single shot visualization. Run it many times. Does it look unbiased?

In [ ]:
arr = list(range(1, 26))
plot_shuffle(arr, bad_shuffle)


## A Real Test of a Real Algorithm

Now let's try the more thorough test. Can you detect any bias?

In [ ]:
arr = list(range(1, 26))
num_shuffles = 10000
plot_colored_histogram(arr, num_shuffles, knuth_shuffle)


## Great Performance

This algorithm performs really well, while giving very well distributed shuffles.

## Exercise

Reflect on what you've learned in this box. Enjoy!